# GPU gauge transformation

### Set parametres and eviroment variables

In [1]:
import numpy as np

# hbar * c [GeV * fm]
hbarc = 0.197326 

# Simulation box 
L = 2      
# N = 128   
N = 64 
tau_sim = 1     
DTS = 8     

# Glasma fields
su_group = 'su3'
Qs = 2        
ns = 50    
factor = 0.8        
g2mu = Qs / factor     
g = np.pi * np.sqrt(1 / np.log(Qs / 0.2))          		
mu = g2mu / g**2          	
ir = 0.1 * g**2 * mu         
uv = 10.0       

# TODO: Run more events
nevents = 1

In [2]:
import os

# I need to add this line to ask resources from a specific GPU, which is free. Our GPU server has no queing system
os.environ["CUDA_VISIBLE_DEVICES"]="3"

# os.environ["MY_NUMBA_TARGET"] = "numba"
os.environ["MY_NUMBA_TARGET"] = "cuda"
os.environ["PRECISION"] = "double"
os.environ['GAUGE_GROUP'] = su_group

# Import relevant modules
import sys
sys.path.append('..')

# Glasma modules
import curraun.core as core
import curraun.mv as mv
import curraun.initial as initial
initial.DEBUG = False

import curraun.su as su
from curraun.numba_target import use_cuda
if use_cuda:
    from numba import cuda

import curraun.su as su
import curraun.lc_gauge as lc_gauge

Using CUDA
Using SU(3)
Using double precision


### We define the simulation routine

In [3]:
# import pickle
from tqdm import tqdm

# Simulation routine
def simulate(): 
    # Derived parameters
    a = L / N
    E0 = N / L * hbarc
    DT = 1.0 / DTS
    maxt = int(tau_sim / a * DTS)

    # Initialize Glasma fields
    s = core.Simulation(N, DT, g)
    va = mv.wilson(s, mu=mu / E0, m=ir / E0, uv=uv / E0, num_sheets=ns)
    vb = mv.wilson(s, mu=mu / E0, m=ir / E0, uv=uv / E0, num_sheets=ns)
    initial.init(s, va, vb)

    # Initialize LC gauge transformation
    lc = lc_gauge.LCGaugeTransf(s)

    # Plus gauge link
    # uplus_lc = su.GROUP_TYPE(np.zeros((maxt//DTS-1, N*N, su.GROUP_ELEMENTS)))

    with tqdm(total=maxt) as pbar:
        for t in range(maxt):            
            # Evolve Glasma fields
            core.evolve_leapfrog(s)

            if t%DTS == 0:
                print('t=', t)
                lc.initialize_vlc()

                # I checked that vlc0 is correctly initialized an swaped with vlc1
                # vlc0 = lc.vlc0.copy()
                # print('vlc0=', vlc0[0, :])
                # print('vlc0=',vlc0[7, :])

                # for xplus in range(maxt//DTS):
                for xplus in range(maxt//DTS):
                    print('xplus=', xplus)
                    lc.evolve_lc(xplus)
                    if xplus > t:
                        print(xplus)
                        vlc0 = lc.vlc0.copy()
                        print('vlc0=', vlc0[7, :])
                        vlc1 = lc.vlc1.copy()
                        print('vlc1=', vlc1[7, :])
                    # elif xplus == (t//DTS) and xplus != 0:
                    # if xplus == t and xplus != 0:
                    #     print(xplus)
                    #     # uplus_lc = lc.up_lc.copy()
                    #     # print("uplus_lc=", uplus_lc[7, :])
                    #     uplus_temp = lc.up_temp.copy()
                    #     print("uplus_temp=", uplus_temp[7, :])


            pbar.update(1)

    if use_cuda:
        cuda.current_context().deallocations.clear()

    # return uplus_lc

In [8]:
a = L / N
E0 = N / L * hbarc
DT = 1.0 / DTS
maxt = int(tau_sim / a * DTS)
maxt//DTS

32

In [4]:
# Supress various horribly long warnings
import warnings
warnings.filterwarnings('ignore')

In [5]:
# uplus_lc = simulate()
simulate()

  0%|          | 0/256 [00:00<?, ?it/s]

t= 0
xplus= 0
xplus= 1
1
vlc0= [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
vlc1= [1.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j]
xplus= 2


  0%|          | 1/256 [00:05<25:00,  5.89s/it]

2
vlc0= [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
vlc1= [ 0.92059599+0.33744686j -0.12857219+0.08218909j -0.0246259 +0.12140983j
  0.11418424+0.08013812j  0.94768953-0.26269779j -0.08827075+0.07498366j
 -0.00337006+0.1384237j   0.09402326+0.02735336j  0.98235312-0.07887516j]
xplus= 3
3
vlc0= [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
vlc1= [ 0.89814231+0.26823124j -0.18311996+0.10279296j  0.2500726 +0.12147746j
  0.21334428+0.10807674j  0.94677825-0.13050883j -0.16217717+0.05550307j
 -0.22862463+0.10920993j  0.20076851-0.04659488j  0.93552528-0.13460072j]
xplus= 4
4
vlc0= [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
vlc1= [ 0.8108506 +0.439058j   -0.07271704+0.06045899j  0.37188951+0.05004518j
  0.17571271+0.07859136j  0.91915375-0.14377248j -0.28875601+0.11855072j
 -0.33518873+0.0186103j   0.34760122-0.0686746j   0.82594968-0.28207533j]
xplus= 5
5
vlc0= [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
vlc

  4%|▎         | 9/256 [00:06<02:03,  2.00it/s]

...

t= 216
xplus= 0
xplus= 1
xplus= 2
xplus= 3
xplus= 4
xplus= 5
xplus= 6
xplus= 7
xplus= 8
xplus= 9
xplus= 10
xplus= 11
xplus= 12
xplus= 13
xplus= 14
xplus= 15
xplus= 16
xplus= 17
xplus= 18
xplus= 19
xplus= 20
xplus= 21
xplus= 22
xplus= 23
xplus= 24
xplus= 25
xplus= 26
xplus= 27
xplus= 28
xplus= 29
xplus= 30
xplus= 31
t= 224
xplus= 0
xplus= 1
xplus= 2
xplus= 3
xplus= 4
xplus= 5
xplus= 6
xplus= 7
xplus= 8
xplus= 9
xplus= 10
xplus= 11
xplus= 12
xplus= 13
xplus= 14
xplus= 15
xplus= 16
xplus= 17
xplus= 18
xplus= 19
xplus= 20
xplus= 21
xplus= 22
xplus= 23
xplus= 24
xplus= 25
xplus= 26
xplus= 27
xplus= 28
xplus= 29
xplus= 30
xplus= 31


 93%|█████████▎| 239/256 [00:09<00:00, 69.90it/s]

t= 232
xplus= 0
xplus= 1
xplus= 2
xplus= 3
xplus= 4
xplus= 5
xplus= 6
xplus= 7
xplus= 8
xplus= 9
xplus= 10
xplus= 11
xplus= 12
xplus= 13
xplus= 14
xplus= 15
xplus= 16
xplus= 17
xplus= 18
xplus= 19
xplus= 20
xplus= 21
xplus= 22
xplus= 23
xplus= 24
xplus= 25
xplus= 26
xplus= 27
xplus= 28
xplus= 29
xplus= 30
xplus= 31
t= 240
xplus= 0
xplus= 1
xplus= 2
xplus= 3
xplus= 4
xplus= 5
xplus= 6
xplus= 7
xplus= 8
xplus= 9
xplus= 10
xplus= 11
xplus= 12
xplus= 13
xplus= 14
xplus= 15
xplus= 16
xplus= 17
xplus= 18
xplus= 19
xplus= 20
xplus= 21
xplus= 22
xplus= 23
xplus= 24
xplus= 25
xplus= 26
xplus= 27
xplus= 28
xplus= 29
xplus= 30
xplus= 31


100%|██████████| 256/256 [00:09<00:00, 26.92it/s]

t= 248
xplus= 0
xplus= 1
xplus= 2
xplus= 3
xplus= 4
xplus= 5
xplus= 6
xplus= 7
xplus= 8
xplus= 9
xplus= 10
xplus= 11
xplus= 12
xplus= 13
xplus= 14
xplus= 15
xplus= 16
xplus= 17
xplus= 18
xplus= 19
xplus= 20
xplus= 21
xplus= 22
xplus= 23
xplus= 24
xplus= 25
xplus= 26
xplus= 27
xplus= 28
xplus= 29
xplus= 30
xplus= 31
